In [2]:
import pyaudio

import window
import numpy as np
import scipy.fftpack

from IPython.display import clear_output

WIN_WIDTH = 512

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 12250

xoff = 0

draw = np.zeros([512, WIN_WIDTH])
def callback(in_data, frame_count, time_info, status):
    global draw, xoff
    
    a = np.fromstring(in_data, dtype=np.int16)
    out = fft(a)
    
    for i in range(min(draw.shape[0], len(out))):
        draw[i, xoff] = out[i]
        
    xoff -= 1
    if xoff <= 0:
        xoff = WIN_WIDTH-1
    
    """
    for i in range(CHUNK):
        val = float((a[i]+4096) / 32)
        draw[max(min(int(val), 255), 0), i] = 1"""
    
    return (None, pyaudio.paContinue)

def fft(signal):
    # Number of samplepoints
    N = CHUNK
    yf = scipy.fftpack.fft(signal)
    
    return 2.0/N * np.abs(yf[:N//2])


p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK,
                stream_callback=callback)

w = window.Window(width=draw.shape[1], height=draw.shape[0])

running = True
while running:
    w.clear()
    
    if draw is not None:
        w.draw_matrices([np.flipud(draw)/200.0 - .5], x=0, y=w.height)
    
    running = w.update()

stream.stop_stream()
stream.close()
p.terminate()